In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import OrdinalEncoder
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split


In [2]:
df = pd.read_excel("../data/sods_limpo.xlsx")

In [3]:
df.head(len(df))

,idade,genero,etnia,pcd,vive_no_brasil,estado_moradia,nivel_ensino,formacao,tempo_experiencia_dados,linguagens_preferidas,bancos_de_dados,cloud_preferida,cargo
0,39,Masculino,Parda,Não,True,Distrito Federal (DF),Pós-graduação,Computação / Engenharia de Software / Sistemas...,de 1 a 2 anos,r,"postgresql, oracle, mysql, sql server",Amazon Web Services (AWS),DBA/Administrador de Banco de Dados
1,53,Masculino,Branca,Não,True,Distrito Federal (DF),Pós-graduação,Computação / Engenharia de Software / Sistemas...,de 3 a 4 anos,python,"postgresql, mysql, oracle, db2",Amazon Web Services (AWS),Desenvolvedor/ Engenheiro de Software/ Analist...
2,27,Masculino,Branca,Não,True,Minas Gerais (MG),Doutorado ou Phd,Estatística/ Matemática / Matemática Computaci...,de 4 a 6 anos,python,google bigquery,Não sei opinar,Cientista de Dados/Data Scientist
3,46,Feminino,Branca,Não,True,Pará (PA),Pós-graduação,Computação / Engenharia de Software / Sistemas...,Não tenho experiência na área de dados,python,microsoft access,Amazon Web Services (AWS),Desenvolvedor/ Engenheiro de Software/ Analist...
4,32,Feminino,Parda,Não,True,Ceará (CE),Pós-graduação,Ciências Biológicas/ Farmácia/ Medicina/ Área ...,Não tenho experiência na área de dados,python,google bigquery,Google Cloud (GCP),Professor
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2976,28,Masculino,Branca,Não,True,Paraná (PR),Graduação/Bacharelado,Computação / Engenharia de Software / Sistemas...,de 3 a 4 anos,julia,sqlite,Azure (Microsoft),Analista de Marketing
2977,26,Masculino,Branca,Não,True,São Paulo (SP),Graduação/Bacharelado,Computação / Engenharia de Software / Sistemas...,de 3 a 4 anos,python,"postgresql, oracle, mariadb, hbase, mysql",Não sei opinar,Analista de Dados/Data Analyst
2978,30,Feminino,Branca,Não,True,Minas Gerais (MG),Mestrado,Outras Engenharias,de 1 a 2 anos,python,"sql server, sap hana",Azure (Microsoft),Engenheiro de Dados/Arquiteto de Dados/Data En...
2979,31,Masculino,Branca,Não,True,São Paulo (SP),Não tenho graduação formal,Prefiro não informar,Menos de 1 ano,python,"microsoft access, oracle",Azure (Microsoft),Outra Opção


## Extraindo Informações Sobre As Variaveis Categoricas

In [4]:
df.select_dtypes(include='object').describe()

,genero,etnia,pcd,estado_moradia,nivel_ensino,formacao,tempo_experiencia_dados,linguagens_preferidas,bancos_de_dados,cloud_preferida,cargo
count,2981,2981,2981,2981,2981,2981,2981,2981,2981,2981,2981
unique,3,7,3,23,7,10,7,21,1429,5,18
top,Masculino,Branca,Não,São Paulo (SP),Graduação/Bacharelado,Computação / Engenharia de Software / Sistemas...,de 1 a 2 anos,python,google bigquery,Amazon Web Services (AWS),Analista de Dados/Data Analyst
freq,2212,1917,2927,1220,1143,1102,906,2581,227,1153,639


## Variáveis Categoricas do dataframe 

    - nivel_ensino (ordinal) == usar ordinal enconding 
    - tempo_de_experiencia (ordinal) == ordinal enconding
    - genero (nominal) == oneHotEnconder 
    - etnia (nominal) == OneHotEnconder 
    - estado_moradia (nominal) == OnehotEnconder
    - pcd e vive_no_brasil (binaria) = mapeamento para 0 || 1
    
    - linguagem_preferidas e bancos_de_dados (OneHotEncoder com CountVectorizer)

    - cargo (Vai ser meu target)
    

Analisando os valores unicos da coluna "estado_moradia", pois é uma das colunas com mais valores unicos e mapeando a fim de diminuir o numero de novas colunas criadas com o OneHotEnconder

In [5]:
df['estado_moradia'].unique()

array(['Distrito Federal (DF)', 'Minas Gerais (MG)', 'Pará (PA)',
       'Ceará (CE)', 'São Paulo (SP)', 'Rio Grande do Sul (RS)',
       'Bahia (BA)', 'Rio de Janeiro (RJ)', 'Mato Grosso do Sul (MS)',
       'Paraná (PR)', 'Espírito Santo (ES)', 'Paraíba (PB)',
       'Não Informado', 'Mato Grosso (MT)', 'Santa Catarina (SC)',
       'Sergipe (SE)', 'Rio Grande do Norte (RN)', 'Pernambuco (PE)',
       'Alagoas (AL)', 'Goiás (GO)', 'Maranhão (MA)', 'Amazonas (AM)',
       'Piauí (PI)'], dtype=object)

In [6]:
# Dicionário para mapear cada estado para sua respectiva região
mapa_regioes = {
    'São Paulo (SP)': 'Sudeste', 'Rio de Janeiro (RJ)': 'Sudeste', 'Minas Gerais (MG)': 'Sudeste',
    'Espírito Santo (ES)': 'Sudeste', 'Paraná (PR)': 'Sul', 'Rio Grande do Sul (RS)': 'Sul',
    'Santa Catarina (SC)': 'Sul', 'Goiás (GO)': 'Centro-Oeste', 'Mato Grosso (MT)': 'Centro-Oeste',
    'Mato Grosso do Sul (MS)': 'Centro-Oeste', 'Distrito Federal (DF)': 'Centro-Oeste',
    'Bahia (BA)': 'Nordeste', 'Sergipe (SE)': 'Nordeste', 'Alagoas (AL)': 'Nordeste',
    'Pernambuco (PE)': 'Nordeste', 'Paraíba (PB)': 'Nordeste', 'Rio Grande do Norte (RN)': 'Nordeste',
    'Ceará (CE)': 'Nordeste', 'Piauí (PI)': 'Nordeste', 'Maranhão (MA)': 'Nordeste',
    'Amazonas (AM)': 'Norte', 'Pará (PA)': 'Norte', 'Acre (AC)': 'Norte', 'Roraima (RR)': 'Norte',
    'Rondônia (RO)': 'Norte', 'Amapá (AP)': 'Norte', 'Tocantins (TO)': 'Norte', 'Não Informado': 'Não Informado'
}
# Criar a nova coluna 'regiao_moradia'

df['regiao_moradia'] = df['estado_moradia'].map(mapa_regioes)

# Remover a coluna original 'estado_moradia'
df = df.drop('estado_moradia', axis=1)

## Realizando o OneHotEncoder na coluna nova "regiao_moradia"

In [7]:
#Criando o Encoder com sparse em False para ver a totalidade da tabela e handle "ignore" para ignorar qualquer nova futura categoria
encoder = OneHotEncoder(sparse_output=False, handle_unknown='ignore')

coluna_para_codificar = df[['regiao_moradia']]

encoded_data = encoder.fit_transform(coluna_para_codificar)

novos_nomes_colunas = encoder.get_feature_names_out(['regiao_moradia'])

encoded_df = pd.DataFrame(encoded_data, columns=novos_nomes_colunas, index=df.index)

df = pd.concat([df.drop('regiao_moradia', axis=1), encoded_df], axis=1)


## Realizando o Mapeamento Binário

In [8]:
# Mapeia 'Sim' para 1 e 'Não' para 0
df['pcd'] = df['pcd'].map({'Sim': 1, 'Não': 0})

# Converte True para 1 e False para 0
df['vive_no_brasil'] = df['vive_no_brasil'].astype(int)

## Realizando o Ordinal Encoder

In [9]:
ordem_nivel_ensino = [
    'Prefiro não informar',
    'Não tenho graduação formal',
    'Estudante de Graduação',
    'Graduação/Bacharelado',
    'Pós-graduação',
    'Mestrado',
    'Doutorado ou Phd'
]

ordem_tempo_experiencia = [
    'Não tenho experiência na área de dados',
    'Menos de 1 ano',
    'de 1 a 2 anos',
    'de 3 a 4 anos',
    'de 4 a 6 anos',
    'de 7 a 10 anos',
    'Mais de 10 anos'
]

# Colunas que vamos transformar
colunas_para_encoder = ['nivel_ensino', 'tempo_experiencia_dados']

# 2. Criar a instância do encoder com as listas de ordem corretas
ordinal_encoder = OrdinalEncoder(
    categories=[ordem_nivel_ensino, ordem_tempo_experiencia],
    dtype=int
)

# 3. Aplicar o encoder para transformar as colunas de texto em números
df[colunas_para_encoder] = ordinal_encoder.fit_transform(df[colunas_para_encoder])

## Realizando o OneHotEncoder

In [10]:
mapa_formacao = {
    'Computação / Engenharia de Software / Sistemas de Informação/ TI': 'Tecnologia',
    'Outras Engenharias': 'Engenharias',
    'Economia/ Administração / Contabilidade / Finanças/ Negócios': 'Negócios e Finanças',
    'Estatística/ Matemática / Matemática Computacional/ Ciências Atuariais': 'Ciências Exatas',
    'Outra opção': 'Outras',
    'Marketing / Publicidade / Comunicação / Jornalismo': 'Comunicação e Marketing',
    'Prefiro não informar': 'Não Informado',
    'Química / Física': 'Ciências Exatas',
    'Ciências Biológicas/ Farmácia/ Medicina/ Área da Saúde': 'Saúde',
    'Ciências Sociais': 'Humanas e Sociais'
}

# 2. Criar a nova coluna 'area_formacao' usando o mapeamento
df['area_formacao'] = df['formacao'].map(mapa_formacao)

# 3. Remover a coluna original 'formacao'
df = df.drop('formacao', axis=1)

In [11]:
encoder = OneHotEncoder(sparse_output=False, handle_unknown='ignore')

colunas_OneHot = ['genero','etnia','area_formacao','cloud_preferida']

encoded_data = encoder.fit_transform(df[colunas_OneHot])

new_column_names = encoder.get_feature_names_out(colunas_OneHot)

encoded_df = pd.DataFrame(encoded_data, columns=new_column_names, index=df.index)

df = pd.concat([df.drop(colunas_OneHot, axis=1), encoded_df], axis=1)


## Realizando o OneHotEnconde usando o CountVectorizer

descrição: utiliza-se o CountVectorizer para realizar o OneHot quando se tem texto e é necessário quebra-lo pra forma novas colunas individuais

In [12]:
df['linguagens_preferidas'] = df['linguagens_preferidas'].fillna('')

#Inicializar o CountVectorizer com o token_pattern CORRIGIDO
#O padrão r'(?u)\b\w+\b' captura "palavras" de 1 ou mais caracteres, incluindo 'r'
vectorizer = CountVectorizer(
    binary=True,
    token_pattern=r'(?u)\b\w+\b'
)

#Aplicar o vectorizer à coluna para aprender o vocabulário e transformar os dados
linguagens_vetorizadas = vectorizer.fit_transform(df['linguagens_preferidas'])

#Criar um novo DataFrame com as colunas para cada linguagem
linguagens_df = pd.DataFrame(
    linguagens_vetorizadas.toarray(),
    columns=vectorizer.get_feature_names_out(),
    index=df.index
)

df = pd.concat([df.drop('linguagens_preferidas', axis=1), linguagens_df], axis=1)


In [13]:
df['bancos_de_dados'] = df['bancos_de_dados'].fillna('')

# 2. Definir a função para quebrar o texto apenas pela vírgula
custom_tokenizer = lambda text: [item.strip() for item in text.split(',')]

# 3. Inicializar o CountVectorizer SEM o 'token_pattern'
vectorizer_db = CountVectorizer(
    binary=True,
    tokenizer=custom_tokenizer 
)

# 4. Aplicar o vectorizer e continuar o processo
db_vetorizados = vectorizer_db.fit_transform(df['bancos_de_dados'])

bancos_df = pd.DataFrame(
    db_vetorizados.toarray(),
    columns=vectorizer_db.get_feature_names_out(),
    index=df.index
)

df = pd.concat([df.drop('bancos_de_dados', axis=1), bancos_df], axis=1)


c:\Users\morae\workspace\jornada\censo_preditivo\venv_novo\Lib\site-packages\sklearn\feature_extraction\text.py:517: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


## Normalizando Dados e comparativo estatistico da coluna idade antes e depois da normalização 

In [14]:
idade_antes_normalizada = df['idade'].describe()

In [15]:
colunas_para_normalizar = ['idade', 'nivel_ensino', 'tempo_experiencia_dados']

scaler = MinMaxScaler()

df[colunas_para_normalizar] = scaler.fit_transform(df[colunas_para_normalizar])

In [16]:
idade_depois_normalizada = df['idade'].describe()

In [17]:
matriz_comparativa = pd.DataFrame({
    'Antes da Normalizacao (Idade em anos)': idade_antes_normalizada,
    'Depois da Normalizacao (0 a 1)': idade_depois_normalizada
})

matriz_comparativa

,Antes da Normalizacao (Idade em anos),Depois da Normalizacao (0 a 1)
count,2981.000000,2981.000000
mean,30.334452,0.342624
std,6.439328,0.178870
min,18.000000,0.000000
25%,26.000000,0.222222
50%,29.000000,0.305556
75%,34.000000,0.444444
max,54.000000,1.000000


In [18]:
df

,idade,pcd,vive_no_brasil,nivel_ensino,tempo_experiencia_dados,cargo,regiao_moradia_Centro-Oeste,regiao_moradia_Nordeste,regiao_moradia_Norte,regiao_moradia_Não Informado,...,sql,sql dbx,sql server,sqlite,superset,sybase,synapse,teradata,vertica,webscraping
0,0.583333,0.0,1,0.666667,0.333333,DBA/Administrador de Banco de Dados,1.0,0.0,0.0,0.0,...,0,0,1,0,0,0,0,0,0,0
1,0.972222,0.0,1,0.666667,0.500000,Desenvolvedor/ Engenheiro de Software/ Analist...,1.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
2,0.250000,0.0,1,1.000000,0.666667,Cientista de Dados/Data Scientist,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
3,0.777778,0.0,1,0.666667,0.000000,Desenvolvedor/ Engenheiro de Software/ Analist...,0.0,0.0,1.0,0.0,...,0,0,0,0,0,0,0,0,0,0
4,0.388889,0.0,1,0.666667,0.000000,Professor,0.0,1.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2976,0.277778,0.0,1,0.500000,0.500000,Analista de Marketing,0.0,0.0,0.0,0.0,...,0,0,0,1,0,0,0,0,0,0
2977,0.222222,0.0,1,0.500000,0.500000,Analista de Dados/Data Analyst,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
2978,0.333333,0.0,1,0.833333,0.333333,Engenheiro de Dados/Arquiteto de Dados/Data En...,0.0,0.0,0.0,0.0,...,0,0,1,0,0,0,0,0,0,0
2979,0.361111,0.0,1,0.166667,0.166667,Outra Opção,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0


In [19]:
df.to_excel('sods_preparado.xlsx', index=False)

## Usando o train_test_split

In [20]:
X = df.drop('cargo', axis=1) 
y = df['cargo']

X_train, X_test, y_train, y_test = train_test_split(
    X, y, 
    test_size=0.2, 
    random_state=42,
    stratify=y
)